In [0]:
from datetime import datetime
from datetime import timedelta
import pandas as pd
import psycopg2
import os
import time
import smtplib as smtp
from openpyxl import load_workbook
import xlsxwriter
from email.mime.multipart import MIMEMultipart
from email.mime.application import MIMEApplication
from email.mime.text import MIMEText
from email.utils import formataddr
from email.header import Header


def create_cancels_file():

    os.chdir(r'/Workspace/Ops Analysis/Reports/circular held flights/circular')
    file_path = os.getcwd()


    Date_circular=(datetime.today()-timedelta(days=1)).strftime('%m%d%y')
    yesterday =(datetime.today()-timedelta(days=1)).strftime('%A, %B %d, %Y')

    # Preparing Cancellation file

    scope = "cbs-udh-databricks-ops-analysis-reports-redshift-udh-prod"

    conn = psycopg2.connect(
        user=dbutils.secrets.get(scope, "user"),
        password=dbutils.secrets.get(scope, "password"),
        host=dbutils.secrets.get(scope, "host"),
        dbname=dbutils.secrets.get(scope, "database"),
        port=dbutils.secrets.get(scope, "port")
    )

    cursor = conn.cursor()

    Summary_circular_holds=pd.read_sql(f'''  with flt_lvl as 
    (
    SELECT AL1.COMPANY_ID,
	 AL5.cir_hld_min as hold_mins,
	 case when AL1.COMPANY_ID in ('UA','CO') then 'Mainline' else 'Express' end as ml_exp,
	 CASE WHEN hold_mins >0 THEN 1 ELSE 0 END AS hold_flights_indicator,
	 case when (AL1.ARRV_STA_CD <> AL1.SCH_ARRV_STA_CD and AL6.divert_reason_cd <> '' and AL1.DPRT_STA_CD = AL1.SCH_DPRT_STA_CD) then 1 else 0 end as Divert_indicator,
	 case when (AL1.ARRV_STA_CD <> AL1.SCH_ARRV_STA_CD and AL6.divert_reason_cd <> '' and AL1.DPRT_STA_CD = AL1.SCH_DPRT_STA_CD) then AL5.cir_hld_min else 0 end as Divert_min,
	 sum(case when (AL1.ARRV_STA_CD <> AL1.SCH_ARRV_STA_CD and AL6.divert_reason_cd <> '' and AL1.DPRT_STA_CD = AL1.SCH_DPRT_STA_CD) then 1 else 0 end)
	 over (partition by AL1.FLT_NUM,AL1.REPORT_DT, AL1.SCH_DPRT_STA_CD,AL1.SCH_ARRV_STA_CD) as flts_diverted,
	 AL1.SCH_ARRV_STA_CD as Sch_arrival_station,
	 
	 AL5.cir_hld_cnt AS cir_hold_count,
	 AL1.FLT_NUM, AL1.REPORT_DT, AL1.SCH_DPRT_STA_CD,
	 AL1.DPRT_STA_CD, AL1.ARRV_STA_CD, 
	 AL1.SCH_DPRT_DTML, AL1.ACT_DPRT_DTML, AL1.OFF_DTML, AL1.ON_DTML, 
	 AL1.SCH_ARRV_DTML, AL1.ACT_ARRV_DTML, AL1.DPRT_DIFF_MINUTES, 
	 AL1.ARRV_DIFF_MINUTES, AL1.DIVERT_TML, AL1.DIVERT_STATUS, AL1.DIVERT_STA_CD, 
	 AL1.DIVERT_DTML, AL1.DIVERT_DTL, AL1.SCH_FLEET_TYPE_DSCR, 
	 AL1.ACT_FLEET_TYPE_DSCR, AL1.SCH_DPRT_TML, AL1.RETURN_TYPE_CD, AL1.SCH_BLK_MINUTES, 
	 AL1.ACT_BLK_MINUTES, AL1.BLK_DIFF_MINUTES, AL1.ACT_TAXI_OUT_MINUTES, AL1.ACT_TAXI_IN_MINUTES, 
	 AL1.FP_AIRBORNE_MINUTES, AL1.ACT_AIRBORNE_MINUTES, AL1.SCH_DPRT_DTL, AL1.ACARS_ARRV_FUEL_LB, 
	 AL1.ACT_AC_CAP, AL1.REV_PAX_ONB, AL1.NRPS_PAX_ONB, AL7.pln_extra_fwt, AL1.NRSA_PAX_ONB, 
	 AL5.fa_flt_url, AL6.divert_tml, AL6.dvrt_sta_cd, AL6.divert_on_tml, AL6.divert_out_tml,
	 AL6.divert_off_tml, AL6.cause, AL6.divert_reason_cd, AL7.pln_contgy_fwt, AL7.pln_contgy_fuel_min as contgy_fuel, AL7.pln_extra_fuel_min as extra_fuel, 
	 AL7.pln_perf_fwt, AL7.pln_perf_fuel_min 
	 FROM co_prod_vmdb.flt_aware AL5, CO_PROD_VMDB.RTF_FLT_LEG_OPERATION_SS AL1 
	 LEFT OUTER JOIN co_prod_vmdb.vw_rtf_divert AL6 ON (AL1.FLT_NUM=AL6.flt_num AND AL1.REPORT_DT=AL6.report_dt AND AL1.DPRT_STA_CD=AL6.dprt_sta_cd) 
	 LEFT OUTER JOIN co_prod_vmdb.pfl_fwz_flt_pln AL7 ON (AL7.oper_flt_nbr=AL1.FLT_NUM AND AL7.sched_flt_orig_dt=AL1.REPORT_DT AND AL7.act_iata_orig_arpt_id=AL1.DPRT_STA_CD)
	 WHERE (AL1.FLT_NUM=AL5.flt_num AND AL1.DPRT_STA_CD=AL5.orig_sta_cd AND AL1.REPORT_DT=AL5.sch_dprt_dtl)  
	 AND ((AL1.REPORT_DT= CURRENT_DATE -1
	 AND (NOT AL1.COMPANY_ID='BU') AND AL1.SCH_CD IN ('S', 'U') AND AL1.RETURN_TYPE_CD=' '))
	 and AL5.cir_hld_min>0
	 
     )

    select 
    ml_exp,
    Sch_arrival_station,
    sum(hold_flights_indicator) as no_of_hold_flights,
    sum(hold_mins)/NULLIF(sum(hold_flights_indicator),0) as avg_hold_mins,
    min(hold_mins) as min_hold_mins,
    max(hold_mins) as max_hold_mins, 
    ROUND((COALESCE (sum(contgy_fuel),0) + COALESCE(SUM(extra_fuel),0))/NULLIF(sum(hold_flights_indicator),0),0) AS avg_planned_hold_mins,
    sum(Divert_indicator) as no_of_diverts,
    sum(Divert_min) as total_hold_diver_mins,
    COALESCE(sum(Divert_min)/NULLIF(sum(Divert_indicator),0),0) AS avg_divert_hold_mins
    from

    (
    select * from flt_lvl
    where flts_diverted=0
    union all
    select * from flt_lvl 
    where flts_diverted>0 and Divert_indicator=1
    )final_data

    group by 1,2
    ORDER BY  ml_exp DESC, no_of_hold_flights DESC 
                            '''
                        ,conn)


    Data_circular_holds=pd.read_sql(f''' 
    with flt_lvl as 
    (
    SELECT AL1.COMPANY_ID,
	 AL5.cir_hld_min as hold_mins,
	 AL1.SCH_ARRV_STA_CD as Sch_arrival_station,
	 AL5.cir_hld_cnt AS cir_hold_count,
	 AL1.FLT_NUM, AL1.REPORT_DT, AL1.SCH_DPRT_STA_CD,
	 AL1.DPRT_STA_CD, AL1.ARRV_STA_CD, 
	 AL1.SCH_DPRT_DTML, AL1.ACT_DPRT_DTML,
	 AL1.SCH_ARRV_DTML, AL1.ACT_ARRV_DTML, AL1.DPRT_DIFF_MINUTES, 
	 AL1.ARRV_DIFF_MINUTES,AL1.SCH_FLEET_TYPE_DSCR, 
	 AL1.ACT_FLEET_TYPE_DSCR,AL1.ACT_TAXI_OUT_MINUTES, AL1.ACT_TAXI_IN_MINUTES, 
	 AL1.FP_AIRBORNE_MINUTES, AL1.ACT_AIRBORNE_MINUTES, AL1.ACARS_ARRV_FUEL_LB, 
	 AL1.ACT_AC_CAP, AL1.REV_PAX_ONB, AL1.NRPS_PAX_ONB, AL1.NRSA_PAX_ONB, 
	 AL6.divert_tml, AL6.dvrt_sta_cd, AL6.divert_on_tml, AL6.divert_out_tml,
	 AL6.divert_off_tml, AL6.cause, AL6.divert_reason_cd, AL7.pln_contgy_fwt, AL7.pln_contgy_fuel_min as contgy_fuel, 
     AL7.pln_extra_fwt, AL7.pln_extra_fuel_min as extra_fuel, 
	 AL7.pln_perf_fwt, AL7.pln_perf_fuel_min,
	 case when AL1.COMPANY_ID in ('UA','CO') then 'Mainline' else 'Express' end as ml_exp,
	 CASE WHEN hold_mins >0 THEN 1 ELSE 0 END AS hold_flights_indicator,
     AL5.fa_flt_url,
	 case when (AL1.ARRV_STA_CD <> AL1.SCH_ARRV_STA_CD and AL6.divert_reason_cd <> '' and AL1.DPRT_STA_CD = AL1.SCH_DPRT_STA_CD) then 1 else 0 end as Divert_indicator,
	 case when (AL1.ARRV_STA_CD <> AL1.SCH_ARRV_STA_CD and AL6.divert_reason_cd <> '' and AL1.DPRT_STA_CD = AL1.SCH_DPRT_STA_CD) then AL5.cir_hld_min else 0 end as Divert_min,
	 sum(case when (AL1.ARRV_STA_CD <> AL1.SCH_ARRV_STA_CD and AL6.divert_reason_cd <> '' and AL1.DPRT_STA_CD = AL1.SCH_DPRT_STA_CD) then 1 else 0 end)
	 over (partition by AL1.FLT_NUM,AL1.REPORT_DT, AL1.SCH_DPRT_STA_CD,AL1.SCH_ARRV_STA_CD) as flts_diverted
	 
	 FROM co_prod_vmdb.flt_aware AL5, CO_PROD_VMDB.RTF_FLT_LEG_OPERATION_SS AL1 
	 LEFT OUTER JOIN co_prod_vmdb.vw_rtf_divert AL6 ON (AL1.FLT_NUM=AL6.flt_num AND AL1.REPORT_DT=AL6.report_dt AND AL1.DPRT_STA_CD=AL6.dprt_sta_cd) 
	 LEFT OUTER JOIN co_prod_vmdb.pfl_fwz_flt_pln AL7 ON (AL7.oper_flt_nbr=AL1.FLT_NUM AND AL7.sched_flt_orig_dt=AL1.REPORT_DT AND AL7.act_iata_orig_arpt_id=AL1.DPRT_STA_CD)
	 WHERE (AL1.FLT_NUM=AL5.flt_num AND AL1.DPRT_STA_CD=AL5.orig_sta_cd AND AL1.REPORT_DT=AL5.sch_dprt_dtl)  
	 AND ((AL1.REPORT_DT= CURRENT_DATE -1
	 AND (NOT AL1.COMPANY_ID='BU') AND AL1.SCH_CD IN ('S', 'U') AND AL1.RETURN_TYPE_CD=' '))
	 and AL5.cir_hld_min>0
    )
    select *
    from
    (
    select * from flt_lvl
    where flts_diverted=0
    union all
    select * from flt_lvl 
    where flts_diverted>0 and Divert_indicator=1
    )final_data
     '''
    ,conn)


    #Create a Pandas Excel writer using XlsxWriter as the engine.
    writer = pd.ExcelWriter(file_path+f'/Circular held flights - Yesterday_ {Date_circular}.xlsx', engine="xlsxwriter")
    
    #Summary circular Holds
    sheet_name='Summary'
    Summary_circular_holds.to_excel(writer, sheet_name='Summary', startrow=4, header=True, index=False)
    
    workbook = writer.book
    worksheet = writer.sheets['Summary']
    
    # Make the columns wider for clarity.
    cell_format = workbook.add_format({'bold': True})
    cell_format2 = workbook.add_format({'bold': True, 'font_color': 'white','bg_color':'#4472C4','font_size':12})
    cell_format.set_align('center')
    cell_format2.set_align('center')
    cell_format3 = workbook.add_format({'bold': True})
    cell_format3.set_align('left')
    
    
    worksheet.write('A1','Flights that have circular holds:',cell_format)
    worksheet.write('A2','Based on FlightAware and UDH tables (CO_PROD_VMDB.RTF_FLT_LEG_OPERATION_SS and co_prod_vmdb.pfl_fwz_flt_pln)',cell_format)
    worksheet.write('H4','Diversions (away from scheduled destination - only if held)',cell_format2)
    worksheet.write('A5','Type',cell_format)
    worksheet.write('B5','Scheduled Arrival Station',cell_format)
    worksheet.write('C5','# of flights w/ holds',cell_format)
    worksheet.write('D5','Average hold minutes',cell_format)
    worksheet.write('E5','Minimum hold minutes',cell_format)
    worksheet.write('F5','Maximum hold minutes',cell_format)
    worksheet.write('G5','Average planned hold fuel minutes(mainline only)',cell_format)
    worksheet.write('H5','# of diverts',cell_format)
    worksheet.write('I5','Total hold minutes during divert',cell_format)
    worksheet.write('J5','Average diversion hold minutes',cell_format)

    
    merge_format=workbook.add_format({'bold': True, 'font_color': 'black','bg_color':'#AEAAAA','font_size':20})
    merge_format.set_align('center')
    
    
    worksheet.merge_range('A1:C1','Flights that have circular holds:',cell_format3)
    worksheet.merge_range('A2:H2','Based on FlightAware and UDH tables (CO_PROD_VMDB.RTF_FLT_LEG_OPERATION_SS and co_prod_vmdb.pfl_fwz_flt_pln)',cell_format3)
    
    worksheet.merge_range('H4:J4','Diversions (away from scheduled destination - only if held)',cell_format2)
    worksheet.merge_range('D1:E1',yesterday, cell_format3)
    
    worksheet.set_column(0,9,20)
    worksheet.set_column(10,11,30)
    worksheet.set_column(12,12,20)
    
  
    #Data Flights with hold minutes
    sheet_name='Flights with hold minutes'
    Data_circular_holds.to_excel(writer, sheet_name='Flights with hold minutes', startrow=3, header=True, index=False)
    
    workbook = writer.book
    worksheet = writer.sheets['Flights with hold minutes']
    
    # Make the columns wider for clarity.
    cell_format = workbook.add_format({'bold': True})
    cell_format.set_align('center')
    
    worksheet.write('A1','Individual flights with hold minutes:',cell_format)
    worksheet.write('A2','Based on FlightAware and UDH tables (CO_PROD_VMDB.RTF_FLT_LEG_OPERATION_SS and co_prod_vmdb.pfl_fwz_flt_pln)',cell_format)
    worksheet.write('A4','Carrier',cell_format)
    worksheet.write('B4','Circular hold mins',cell_format)
    worksheet.write('C4','Scheduled Dest',cell_format)
    worksheet.write('D4','Circular Hold Count',cell_format)
    worksheet.write('E4','Flight Number',cell_format)
    worksheet.write('F4','Date',cell_format)
    worksheet.write('G4','Schedule Origin',cell_format)
    worksheet.write('H4','Actual Origin',cell_format)
    worksheet.write('I4','Actual Dest',cell_format)
    worksheet.write('J4','Scheduled Dptr Time',cell_format)
    worksheet.write('K4','Actual Dptr Time',cell_format)
    worksheet.write('L4','Scheduled Arvl Time',cell_format)
    worksheet.write('M4','Actual Arvl Time',cell_format)
    worksheet.write('N4','Dptr Variance',cell_format)
    worksheet.write('O4','Arvl Variance',cell_format)
    worksheet.write('P4','Scheduled Fleet',cell_format)
    worksheet.write('Q4','Actual Fleet',cell_format)
    worksheet.write('R4','Actual  Taxi out',cell_format)
    worksheet.write('S4','Scheduled Taxi in',cell_format)
    worksheet.write('T4','Plan Airborne Mins',cell_format)
    worksheet.write('U4','Actual Airborne Mins',cell_format)
    worksheet.write('V4','Arrival Fuel Pounds',cell_format)
    worksheet.write('W4','Aircraft Capacity',cell_format)
    worksheet.write('X4','Revenue Pax',cell_format)
    worksheet.write('Y4','NPRS Pax',cell_format)
    worksheet.write('Z4','NPSA Pax',cell_format)
    worksheet.write('AA4','Divert Time',cell_format)
    worksheet.write('AB4','Divert Station Code',cell_format)
    worksheet.write('AC4','Divert On Time',cell_format)
    worksheet.write('AD4','Divert OUT Time',cell_format)
    worksheet.write('AE4','Divert OFF Time',cell_format)
    worksheet.write('AF4','Divert Cause',cell_format)
    worksheet.write('AG4','Divert Code',cell_format)
    worksheet.write('AH4','Planned Contingncy fuel weight',cell_format)
    worksheet.write('AI4','Planned Contingency fuel minutes',cell_format)
    worksheet.write('AJ4','Planned Extra fuel weights',cell_format)
    worksheet.write('AK4','Planned Extra fuel minutes',cell_format)
    worksheet.write('AL4','Planned Performance Fuel weight',cell_format)
    worksheet.write('AM4','Planned performance fuel minutes',cell_format)
    worksheet.write('AN4','Type',cell_format)
    worksheet.write('AO4','Flight Hold Indicator',cell_format)
    worksheet.write('AP4','Flt Url',cell_format)
    worksheet.write('AQ4','Divert Indicator',cell_format)
    worksheet.write('AR4','Divert Mins',cell_format)
    worksheet.write('AS4','Flight Diverted',cell_format)
    
    merge_format=workbook.add_format({'bold': True})
    merge_format.set_align('left')
    
    worksheet.merge_range('A1:C1','Individual flights with hold minutes:', merge_format)
    worksheet.merge_range('D1:E1',yesterday, merge_format)
    
    
  
    
    worksheet.set_column(0,9,20)
    worksheet.set_column(10,11,30)
    worksheet.set_column(12,12,20)

 

    writer.close()
    
    return 0

create_cancels_file()



#EMAIL
os.chdir(r'/Workspace/Ops Analysis/Reports/circular held flights/circular')
file_path = os.getcwd()
Date_circular=(datetime.today()-timedelta(days=1)).strftime('%m%d%y')

print('Sending Email')
server = smtp.SMTP('mailout.ual.com:25')
emailFrom = 'harmanpreet.singh@united.com'
emailFromHdr = formataddr((str(Header(r'Singh, Harmanpreet')), emailFrom))


listEmailTo =[ 'Harmanpreet.singh@united.com']
listEmailCc =[]
listEmailBcc =[]





msg = MIMEMultipart()
msg['From'] = emailFromHdr
msg['To'] = ','.join(listEmailTo)
msg['Cc'] = ','.join(listEmailCc)
msg['Bcc'] = ','.join(listEmailBcc)

msg['Subject'] = 'Circular Held Flights- Yesterday'

htmlBdy = """<html>

    	<head></head>
	    <body>
		<p><strong>***This is an automated email, Powered by Databricks*** </strong>
        <br>
		<br>Hi,
        <br>
		<br>Please find attached Circular held Flights for Yesterday
        <br>
        <br>
        <br>Regards,
		<br><font size="3"><strong>Harman</strong>
	    </body>
        </html>"""

xlsx = open(file_path + f'/Circular held flights - Yesterday_ {Date_circular}.xlsx', 'rb').read()
msgXlsx = MIMEApplication(xlsx, 'xlsx')  
msgXlsx.add_header('Content-ID', '<xlsx1>')  
msgXlsx.add_header('Content-Disposition', 'attachment', filename='Circular held flights - Yesterday' + Date_circular + '.xlsx')
msgXlsx.add_header('Content-Disposition', 'inline', filename='Circular held flights - Yesterday' + Date_circular +'.xlsx')

body = MIMEText(htmlBdy, 'html')
msg.attach(body)
msg.attach(msgXlsx)
server.sendmail(emailFrom, listEmailTo + listEmailCc + listEmailBcc, msg.as_string())

server.quit()






In [0]:
dbutils.secrets.get(
    scope="cbs-udh-databricks-ops-analysis-reports-redshift-udh-prod",
    key="user"
)

